In [22]:
import json, os, requests
from requests import request
import pandas as pd
import numpy as np
import time
import pandas_read_xml as pdx
import xmltodict, json
import flatdict

In [2]:

# get data
CIK = '0000320193'
req = request(url=f'https://data.sec.gov/submissions/CIK{CIK}.json',
            method='GET',
            headers={"User-Agent": "Mozilla/5.0"})


In [40]:

#mk raw db
db = pd.DataFrame.from_dict(req.json()['filings']['recent'])

# date filter
maxDate = 2023
minDate = 2022
deleteColums = []

counter = 0
for i in db['filingDate']:
    year = int(i.split('-')[0])
    if year > maxDate or year < minDate and counter not in deleteColums:
        deleteColums.append(counter)
    counter += 1


counter = 0
#rm non-form 4s
for i in db['form']:
     if i != '4'and counter not in deleteColums:
          deleteColums.append(counter)
     counter += 1

counter = 0
#rm xbrl
for i in db['isXBRL']:
     if i == 1 and counter not in deleteColums:
          deleteColums.append(counter)
     counter += 1

db.drop(deleteColums, inplace=True, axis=0)
db

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0000320193-23-000070,2023-05-18,2023-05-16,2023-05-18T18:32:21.000Z,,4,,,,5705,0,0,xslF345X04/wf-form4_168444912415136.xml,FORM 4
1,0000320193-23-000068,2023-05-12,2023-05-10,2023-05-12T18:30:34.000Z,,4,,,,4583,0,0,xslF345X04/wf-form4_168393061463065.xml,FORM 4
2,0000320193-23-000066,2023-05-10,2023-05-08,2023-05-10T18:30:22.000Z,,4,,,,5105,0,0,xslF345X04/wf-form4_168375780902324.xml,FORM 4
9,0000320193-23-000057,2023-04-18,2023-04-15,2023-04-18T18:31:39.000Z,,4,,,,16150,0,0,xslF345X04/wf-form4_168185708431590.xml,FORM 4
10,0000320193-23-000055,2023-04-17,2023-04-13,2023-04-17T18:31:10.000Z,,4,,,,5207,0,0,xslF345X04/wf-form4_168177065948283.xml,FORM 4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,0000320193-22-000020,2022-02-03,2022-02-01,2022-02-03T18:35:50.000Z,,4,,,,9114,0,0,xslF345X03/wf-form4_164393133439643.xml,FORM 4
111,0000320193-22-000019,2022-02-03,2022-02-01,2022-02-03T18:34:28.000Z,,4,,,,7710,0,0,xslF345X03/wf-form4_164393124898522.xml,FORM 4
112,0000320193-22-000018,2022-02-03,2022-02-01,2022-02-03T18:33:24.000Z,,4,,,,7036,0,0,xslF345X03/wf-form4_164393117233036.xml,FORM 4
113,0000320193-22-000017,2022-02-03,2022-02-01,2022-02-03T18:31:58.000Z,,4,,,,7034,0,0,xslF345X03/wf-form4_164393109986560.xml,FORM 4


In [ ]:
# TODO MK a loop for the following code to run through each file with a max of 10 req per sec

In [4]:
# get form data
## warning limit of 10 requests per second 

test = db['accessionNumber'][0].replace('-', '')
test2 = db['primaryDocument'][0].split('/')[1]
test3 = f'https://www.sec.gov/Archives/edgar/data/{CIK}/{test}/{test2}'

fileReq = request(url=test3,
            method='GET',
            headers={"User-Agent": "Mozilla/5.0"})

fileRes = fileReq.text
fileRes

'<?xml version="1.0"?>\n<ownershipDocument>\n\n    <schemaVersion>X0407</schemaVersion>\n\n    <documentType>4</documentType>\n\n    <periodOfReport>2023-05-16</periodOfReport>\n\n    <notSubjectToSection16>0</notSubjectToSection16>\n\n    <issuer>\n        <issuerCik>0000320193</issuerCik>\n        <issuerName>Apple Inc.</issuerName>\n        <issuerTradingSymbol>AAPL</issuerTradingSymbol>\n    </issuer>\n\n    <reportingOwner>\n        <reportingOwnerId>\n            <rptOwnerCik>0001051401</rptOwnerCik>\n            <rptOwnerName>JUNG ANDREA</rptOwnerName>\n        </reportingOwnerId>\n        <reportingOwnerAddress>\n            <rptOwnerStreet1>ONE APPLE PARK WAY</rptOwnerStreet1>\n            <rptOwnerStreet2></rptOwnerStreet2>\n            <rptOwnerCity>CUPERTINO</rptOwnerCity>\n            <rptOwnerState>CA</rptOwnerState>\n            <rptOwnerZipCode>95014</rptOwnerZipCode>\n            <rptOwnerStateDescription></rptOwnerStateDescription>\n        </reportingOwnerAddress>\n 

In [39]:
xml = xmltodict.parse(fileRes)

xml

{'ownershipDocument': {'schemaVersion': 'X0407',
  'documentType': '4',
  'periodOfReport': '2023-05-16',
  'notSubjectToSection16': '0',
  'issuer': {'issuerCik': '0000320193',
   'issuerName': 'Apple Inc.',
   'issuerTradingSymbol': 'AAPL'},
  'reportingOwner': {'reportingOwnerId': {'rptOwnerCik': '0001051401',
    'rptOwnerName': 'JUNG ANDREA'},
   'reportingOwnerAddress': {'rptOwnerStreet1': 'ONE APPLE PARK WAY',
    'rptOwnerStreet2': None,
    'rptOwnerCity': 'CUPERTINO',
    'rptOwnerState': 'CA',
    'rptOwnerZipCode': '95014',
    'rptOwnerStateDescription': None},
   'reportingOwnerRelationship': {'isDirector': '1',
    'isOfficer': '0',
    'isTenPercentOwner': '0',
    'isOther': '0',
    'officerTitle': None,
    'otherText': None}},
  'aff10b5One': '0',
  'nonDerivativeTable': {'nonDerivativeTransaction': {'securityTitle': {'value': 'Common Stock'},
    'transactionDate': {'value': '2023-05-16'},
    'transactionCoding': {'transactionFormType': '4',
     'transactionCode'